<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/master/Class_03_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

**Module 3: Introduction to TensorFlow**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Integrative Biology](https://sciences.utsa.edu/integrative-biology/), [UTSA](https://www.utsa.edu/)


### Module 3 Material


* Part 3.1: Deep Learning and Neural Network Introduction
* Part 3.2: Using Keras to Build Regression Models
* Part 3.3: Using Keras to Build Classification Models
* Part 3.4: Saving and Loading a Keras Neural Network
* **Part 3.5: Early Stopping in Keras to Prevent Overfitting**


### Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [ ]:
# YOU MUST RUN THIS CELL FIRST

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Part 3.5: Early Stopping in Keras to Prevent Overfitting

It can be difficult to determine how many epochs to cycle through to train a neural network. **_Overfitting_** will occur if you train the neural network for too many epochs, and the neural network will not perform well on new data, despite attaining a good accuracy on the training set. Overfitting occurs when a neural network is trained to the point that it begins to memorize rather than generalize, as demonstrated in Figure 3.OVER. 

**Training vs. Validation Error for Overfitting**
![Training vs. Validation Error for Overfitting](https://biologicslab.co/BIO1173/images/class_3_training_val.png "Training vs. Validation Error for Overfitting")

It is important to segment the original dataset into several datasets:

* **Training Set**
* **Validation Set**
* **Holdout Set**

You can construct these sets in several different ways. The following programs demonstrate some of these.

The first method is a training and validation set. We use the training data to train the neural network until the validation set no longer improves. This attempts to stop at a near-optimal training point. This method will only give accurate "out of sample" predictions for the validation set; this is usually 20% of the data. The predictions for the training data will be overly optimistic, as these were the data that we used to train the neural network. Figure 3.VAL demonstrates how we divide the dataset.

**Training with a Validation Set**
![Training with a Validation Set](https://biologicslab.co/BIO1173/images/class_1_train_val.png "Training with a Validation Set")

## Early Stopping

We will now see an example of classification training with early stopping. We will train a neural network until the error no longer improves on the validation set.

### Example 1: Early Stopping with Classification

The code in the cell below builds and trains a **_classification_** neural network called `irisModel`. The model is trained/fitted to the Iris flower dataset (`iris.csv`) downloaded from the course HTTPS server and stored in the DataFrame `irisDF`. 

The independent variables, or X-values, are the values in the columns `sepal_length`, `sepal_width`, `petal_length` and `petal_width`. The values are store in a Numpy array called `irisX`. 

The dependent variable (Y-value) is the column `species` which contains the names of the three Iris species in the dataset, _Iris setosa_, _Iris_ versicolor_ and _Iris virginica_. 

Since the species names are entered in the `species` column as strings, it is necessary to use One-Hot Encoding to convert these strings into the values `0` and `1` using the command `pd.get_dummies()`. The variable holding the dependent values, `irisY`, is created from the `dummies.values` as shown below.

#### Early Stopping

In order to implement _Early Stopping_, it is first necessary to split the dataset into 4 separate groups: X train, X test, Y train and Y test using the function `train_test_split()`. The argument `test_size=0.25` tells the function that 75% of the data should be put into the two train sets (i.e. `irisX_train` and `irisY_train`) and the remaining 25% should be put into the two validation sets `irisX_test` and `irisY_test`. 

Since the separation of data into training and test sets is a random process, the argument `random_state=42` is used for teaching/demonstration purposes to insure that the `split` occurs at the same places when the code is re-run. In normal use, you wouldn't set the random seed.

The model, `irisModel`, is a densely connected sequential neural network with two hidden layers. The 1st layer has 50 neurons, the 2nd hidden layer 25. The activation function for both hidden layers is `relu`. Since this function of this model is classification, the `softmax` activation function is used in the output layer. The model is compiled with the 'categorical_crossentropy` loss function and the `adam` optimizer. 

The code for implementing early stopping variable `irisMoniter` is shown below:

~~~text
# Build monitor for early stopping
irisMonitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
        verbose=1, mode='auto', restore_best_weights=True)
~~~
The meaning/function of the different arguments will be discussed below.

Finally, the model is fitted to the Iris data with the number of epochs set to **1000!** Don't worry, you won't have to wait forever for the training to complete--thanks to early stopping.

In [ ]:
# Example 1: Early stopping with classification

import pandas as pd
import io
import requests
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

# Read dataset and create DataFrame-------------------------------------
irisDF = pd.read_csv("https://biologicslab.co/BIO1173/data/iris.csv",
    na_values=['NA', '?'])

# Create feature vector-------------------------------------------------
# Generate X-values
irisX = irisDF[['sepal_length', 'sepal_width', 
                'petal_length', 'petal_width']].values
irisX = np.asarray(irisX).astype('float32')

# Generate Y-values
dummies = pd.get_dummies(irisDF['species']) # Classification
SpeciesNames = dummies.columns  # List with species names
irisY = dummies.values # Y-values
irisY = np.asarray(irisY).astype('float32')

# Split into validation and training sets------------------------------
irisX_train, irisX_test, irisY_train, irisY_test = train_test_split(    
    irisX, irisY, test_size=0.25, random_state=42)

# Build neural network-------------------------------------------------
irisModel = Sequential()
irisModel.add(Dense(50, input_dim=irisX.shape[1], activation='relu')) 
irisModel.add(Dense(25, activation='relu')) 
irisModel.add(Dense(irisY.shape[1],activation='softmax')) 
irisModel.compile(loss='categorical_crossentropy', optimizer='adam')

# Build monitor for early stopping-------------------------------------
irisMonitor = EarlyStopping(monitor='val_loss', 
              min_delta=1e-3, patience=5, verbose=1, 
              mode='auto', restore_best_weights=True)

# Train model----------------------------------------------------------
irisModel.fit(irisX_train,irisY_train,validation_data=(irisX_test,irisY_test),
        callbacks=[irisMonitor],verbose=2,epochs=1000)


If your code is correct, you should see something similar to the following output:

~~~text
Epoch 1/1000
4/4 - 1s - loss: 1.6055 - val_loss: 1.7102 - 564ms/epoch - 141ms/step
Epoch 2/1000
4/4 - 0s - loss: 1.4318 - val_loss: 1.5122 - 59ms/epoch - 15ms/step
Epoch 3/1000
4/4 - 0s - loss: 1.2826 - val_loss: 1.3403 - 55ms/epoch - 14ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.1618 - val_loss: 1.1992 - 55ms/epoch - 14ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.0673 - val_loss: 1.0937 - 55ms/epoch - 14ms/step

....................................

Epoch 103/1000
4/4 - 0s - loss: 0.0989 - val_loss: 0.0813 - 47ms/epoch - 12ms/step
Epoch 104/1000
4/4 - 0s - loss: 0.0928 - val_loss: 0.0766 - 51ms/epoch - 13ms/step
Epoch 105/1000
4/4 - 0s - loss: 0.0935 - val_loss: 0.0738 - 51ms/epoch - 13ms/step
Epoch 106/1000
Restoring model weights from the end of the best epoch: 101.
4/4 - 0s - loss: 0.0911 - val_loss: 0.0767 - 55ms/epoch - 14ms/step
Epoch 106: early stopping

<keras.callbacks.History at 0x21b49b53220>

~~~

Even though the number of epochs was set to 1000, the training/fitting should have stopped much earlier. For example, on the machine this assigment is being created, the training stopped after only 106 epochs with epoch 101 having the best predictions.

### Arguments that Control the EarlyStopping object

There are a number of parameters (arguments) that are specified to the **EarlyStopping** object. 

* **min_delta** This value should be kept small. It simply means the minimum change in error to be registered as an improvement.  Setting it even smaller will not likely have a great deal of impact.
* **patience** How long should the training wait for the validation error to improve?  
* **verbose** How much progress information do you want?
* **mode** In general, always set this to "auto".  This allows you to specify if the error should be minimized or maximized.  Consider accuracy, where higher numbers are desired vs log-loss/RMSE where lower numbers are desired.
* **restore_best_weights** This should always be set to true.  This restores the weights to the values they were at when the validation set is the highest.  Unless you are manually tracking the weights yourself (we do not use this technique in this course), you should have Keras perform this step for you.

As you can see from above, the entire number of requested epochs were not used.  The neural network training stopped once the validation set no longer improved.

### **Exercise 1: Early Stopping with Classification: Heart Disease data**

In the cell below, write the code to read the Heart Failure dataset ("heart_disease.csv") from the course HTTPS server and store the data in a DataFrame called `hdDF`. 

You can use this code chunk to read the datafile and create your DataFrame:
~~~text
# Read dataset and create DataFrame
hdDF = pd.read_csv(
    "http://biologicslab.co/BIO1173/data/heart_disease.csv",
    na_values=['NA', '?'])
~~~

For you independent variables (X-values)  **only** use the columns Age, RestingBP, Cholesterol, MaxHR and Oldpeak. You should name the Numpy array holding your X-values `hdX`. 

Use the column `HeartDisease` as your dependent variable (Y-values). You will need to One-Hot Encode this column, and use the `dummies.values` as your Y-values, `hfY` as shown in the code chunk below:
~~~text
# Generate Y-values
dummies = pd.get_dummies(hdDF['HeartDisease']) # Classification
DiseaseNames = dummies.columns
hdY = dummies.values  # Y-values
hdY = np.asarray(hdY).astype('float32')
~~~

Use the `train_test_split(hdX, hdY, test_size=0.25, random_state=42)` function to create `hdX_train`, `hdX_test`, `hdY_train` and `hdY_test` datasets. 

Build a Sequential neural network called `hdModel` with 2 hidden layers with 50 neurons in the first layer and 25 neurons in the second layer. Use `relu` activation for these two hidden layers. The output layer should use `softmax` activation. Don't forget that the number of neurons in your Output layers needs to be defined by this variable: `hdY.shape[1]`.

Compile your model using `categorical_crossentropy` as the loss function with `adam` as the optimizer. 

After your model has been compiled, create an object called `hdMonitor` to provide `EarlyStopping()` with the same arguments are shown in Example 1. 

Finally, train your model for 1000 epochs but use your `hdMonitor` to enable early stopping. 

In [ ]:
# Insert your code for Exercise 1 here 



If your code is correct, the training of your `hdModel` neural network should have stopped early, before reaching 30 epochs. 
~~~text
Epoch 15/1000
22/22 - 0s - loss: 0.5458 - val_loss: 0.5953 - 122ms/epoch - 6ms/step
Epoch 16/1000
22/22 - 0s - loss: 0.6511 - val_loss: 0.7310 - 124ms/epoch - 6ms/step
Epoch 17/1000
22/22 - 0s - loss: 0.6141 - val_loss: 0.6277 - 124ms/epoch - 6ms/step
Epoch 18/1000
22/22 - 0s - loss: 0.5492 - val_loss: 0.5794 - 123ms/epoch - 6ms/step
Epoch 19/1000
Restoring model weights from the end of the best epoch: 14.
22/22 - 0s - loss: 0.5356 - val_loss: 0.6046 - 127ms/epoch - 6ms/step
Epoch 19: early stopping

<keras.callbacks.History at 0x21b50503f70>
~~~

In the example shown above, the training stopped after 19 epochs. The minimum `val_loss` occurred after epoch 14, and started to increase due to overfitting.  

### Example 2: Compute Accuracy Score

Let's see what effect early stopping might have on the accuracy of the `irisModel`?

The code below illustrates how to compute the accuracy score for the model `irisModel` created in Example 1 using the Keras `model.predict()` function and the `accuracy_score()` function from the `scikit-learn` metrics package. To keep variable names separate between Examples and Exercises, the prefix `iris` has been added to different variables that are generated. 

In [ ]:
# Example 2: Compute accuracy score

from sklearn.metrics import accuracy_score

irisPred = irisModel.predict(irisX_test)
irisPredict_classes = np.argmax(irisPred,axis=1)
irisExpected_classes = np.argmax(irisY_test,axis=1)
irisCorrect = accuracy_score(irisExpected_classes,irisPredict_classes)
print(f"Accuracy: {irisCorrect}")

If your code is correct you should see something similar to the following output:
~~~text
2/2 [==============================] - 0s 8ms/step
Accuracy: 0.9736842105263158
~~~
WOW! Perfect nearly perfect accuracy!

### **Exercise 2: Compute Accuracy Score**

In the cell below, compute the accuracy score for your `hdModel` and print out the results. Add the prefix `hd` to your variables to keep them from interferring with the variables generated in Example 2 above.

In [ ]:
# Insert your code for Exercise 2 here



If your code is correct you should see something similar to the following output:
~~~text
8/8 [==============================] - 0s 3ms/step
Accuracy: 0.7347826086956522
~~~

According to the output above, your `apModel` is only about 75% accurate when it comes to predicting apple quality. Apparently, it's a harder to predict an apple's `Quality` with a classification neural network than to identify the `species` name of an Iris flower.

## Early Stopping with Regression

The following code demonstrates how we can apply early stopping to a regression problem.  The technique is similar to the early stopping for classification code that we just saw.

### Example 3: Early Stopping with Regression

The regression neural network `petalModel`, constructed in the cell below, is designed to predict the `petal_length`, the Y-value, based on the flower's `sepal_length`, `sepal_width`, `petal_width` and `species`. 

We begin by reading the Iris Flower dataset to create a DataFrame called `petalDF`. 

The next step is to prepare the feature vector. The first step is to take care of columns with non-numeric values. Since the column `species` contains the categorical values `Iris-setosa`, `Iris-versicolor` and `Iris-virginica`, we will map these strings to the integers `1`, `2`, and `3`, respectively. This mapping must be done _before_ the column `species` can be included as part of the X-values. 

Here is the code chunk that generates the X-values and stores them in a Numpy array called `petalX`:
~~~text
# Generate X-values
petalX = petalDF[['sepal_length', 'sepal_width', 
                'petal_width', 'species']].values
petalX = np.asarray(petalX).astype('float32')
~~~

And since we are building a **_regression_** neural network to predict `petal_length`, the numerical values in the column `petal_length` are used _directly_ to generate the Y-values as shown in this code chunk:
~~~text
# Generate Y-values
petalY = petalDF['petal_length'].values # Y-values
petalY = np.asarray(petalY).astype('float32')
~~~~
In other words, you do **not** One-Hot Encode the column with the Y-values.

You should also note that when building a _regression_ neural network there is only one neuron is the output layer and no activation function as show here:

~~~text
petalModel.add(Dense(1)) # Output
~~~

Finally, we need to change the loss function to RMSE when the model is compiled:
~~~text
petalModel.compile(loss='mean_squared_error', optimizer='adam')
~~~


In [ ]:
# Example 3: Early stopping with regression

import pandas as pd
import io
import requests
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

# Read dataset and create DataFrame-------------------------------------
petalDF = pd.read_csv("https://biologicslab.co/BIO1173/data/iris.csv",
    na_values=['NA', '?'])

# Create feature vector-------------------------------------------------

# Define the mapping dictionary
mapping = {'Iris-setosa': 1, 
           'Iris-versiolor': 2,
           'Iris-virginica': 3}
# Map the integer column to strings
petalDF['species'] = petalDF['species'].map(mapping)

# Generate X-values
petalX = petalDF[['sepal_length', 'sepal_width', 
                'petal_width', 'species']].values
petalX = np.asarray(petalX).astype('float32')

# Generate Y-values
petalY = petalDF['petal_length'].values # Y-values
petalY = np.asarray(petalY).astype('float32')

# Split into validation and training sets------------------------------
petalX_train, petalX_test, petalY_train, petalY_test = train_test_split(    
    petalX, petalY, test_size=0.25, random_state=42)

# Build neural network-------------------------------------------------
petalModel = Sequential()
petalModel.add(Dense(50, input_dim=petalX.shape[1], 
                    activation='relu')) # Hidden 1 
petalModel.add(Dense(25, activation='relu')) # Hidden 2
petalModel.add(Dense(1)) # Output
petalModel.compile(loss='mean_squared_error', optimizer='adam')

# Build monitor for early stopping-------------------------------------
petalMonitor = EarlyStopping(monitor='val_loss', 
              min_delta=1e-3, patience=5, verbose=1, 
              mode='auto', restore_best_weights=True)

# Train model----------------------------------------------------------
petalModel.fit(petalX_train,petalY_train,validation_data=(petalX_test,petalY_test),
              callbacks=[irisMonitor],verbose=2,epochs=1000)


If your code is correct, you should see something similar to the following output:
~~~text
Epoch 1/1000
4/4 - 0s - loss: 17.6558 - val_loss: 16.3612 - 420ms/epoch - 105ms/step
Epoch 2/1000
4/4 - 0s - loss: 17.3584 - val_loss: 16.2709 - 62ms/epoch - 16ms/step
Epoch 3/1000
4/4 - 0s - loss: 17.2609 - val_loss: 16.1802 - 62ms/epoch - 16ms/step
Epoch 4/1000
4/4 - 0s - loss: 17.1694 - val_loss: 16.0887 - 62ms/epoch - 16ms/step
Epoch 5/1000
4/4 - 0s - loss: 17.0703 - val_loss: 15.9978 - 66ms/epoch - 16ms/step

.....................................

Epoch 145/1000
4/4 - 0s - loss: 3.0206 - val_loss: 3.3948 - 50ms/epoch - 12ms/step
Epoch 146/1000
4/4 - 0s - loss: 3.0172 - val_loss: 3.3949 - 53ms/epoch - 13ms/step
Epoch 147/1000
4/4 - 0s - loss: 3.0133 - val_loss: 3.3951 - 47ms/epoch - 12ms/step
Epoch 148/1000
Restoring model weights from the end of the best epoch: 143.
4/4 - 0s - loss: 3.0104 - val_loss: 3.3954 - 56ms/epoch - 14ms/step
Epoch 148: early stopping

<keras.callbacks.History at 0x21b45a69d30>
~~~

In this example, training stopped after 148 epochs. The epoch with the lowest loss was epoch 143. 

### **Exercise 3: Early Stopping with Regression**

If the cell below, use the same Apple Quality dataset as used in Example 3 to construct a regression neural network that can predict the 'Acidity' of an apple instead of it `Sweetness`. 

Call your new model `acidModel`, the independent variables `acidX`, the dependent variable `acidY`, `acidX_train` and so forth. As in Example 3, implement early stopping.  

In [ ]:
# Insert your code for Exercise 3 here



If your code is correct, you should see something similar to the following output:
~~~text
Epoch 1/1000
94/94 - 1s - loss: 4.2824 - val_loss: 3.8108 - 865ms/epoch - 9ms/step
Epoch 2/1000
94/94 - 0s - loss: 3.5398 - val_loss: 3.4943 - 400ms/epoch - 4ms/step
Epoch 3/1000
94/94 - 0s - loss: 3.2748 - val_loss: 3.2450 - 404ms/epoch - 4ms/step
Epoch 4/1000
94/94 - 0s - loss: 3.0535 - val_loss: 3.0415 - 403ms/epoch - 4ms/step
Epoch 5/1000
94/94 - 0s - loss: 2.8624 - val_loss: 2.8641 - 435ms/epoch - 5ms/step

..................................

Epoch 51/1000
94/94 - 0s - loss: 1.5606 - val_loss: 1.8228 - 425ms/epoch - 5ms/step
Epoch 52/1000
94/94 - 0s - loss: 1.5516 - val_loss: 1.8347 - 446ms/epoch - 5ms/step
Epoch 53/1000
94/94 - 0s - loss: 1.5503 - val_loss: 1.8448 - 425ms/epoch - 5ms/step
Epoch 54/1000
94/94 - 0s - loss: 1.5404 - val_loss: 1.8277 - 387ms/epoch - 4ms/step
Epoch 55/1000
94/94 - 0s - loss: 1.5272 - val_loss: 1.8321 - 393ms/epoch - 4ms/step
Epoch 56/1000
Restoring model weights from the end of the best epoch: 51.
94/94 - 0s - loss: 1.5240 - val_loss: 1.8226 - 397ms/epoch - 4ms/step
Epoch 56: early stopping

<keras.callbacks.History at 0x21b55ea5dc0>
~~~

In this example, the best epoch was 51.

### Example 4: Compute the RMSE

When working with neural networks that perform a regression analysis, it is customary to use the Root Mean Square Error (RMSE) as a measurement of predictive accuracy. The code in the cell below shows how to compute the RMSE for the `orModel` neural network and then print out the result.

In [ ]:
# Example 4 Compute the RMSE

# Measure RMSE error. 
petalPred = petalModel.predict(petalX_test)
petalScore = np.sqrt(metrics.mean_squared_error(petalPred,petalY_test))

# Print out the results
print(f"Final score (RMSE): {petalScore}")

If your code is correct you should see something similar to the following output:
~~~text
2/2 [==============================] - 0s 3ms/step
Final score (RMSE): 1.8425109386444092
~~~

### **Exercise 4: Compute the RMSE**

In the cell below, write the code to compute RMSE for your neural network model `acidModel` and then print out the results.

In [ ]:
# Insert your code for Exercise 4 here



If your code is correct you should see something similar to the following output:
~~~text
32/32 [==============================] - 0s 2ms/step
Final score (RMSE): 1.3298665285110474
~~~

### Example 5: _Ad Hoc_ Prediction

The code in the cell below uses the `petalModel` to predict the petal length of the first flower in the Iris Flower dataset (`Flower_0`). 

The X-values for `Flower_0` are simply copied from the Numpy array `petalX` using the following code chunk:
~~~text
# Load X-values
Flower_0 = petalX[[0]]
~~~~

The code then uses the `petalModel` to predict what it thinks the `petal_length` should be, based on the flower's `sepal length`, `sepal_width`, `petal_width`, `species`. This prediction is stored in `petalPred`. 

Finally, the code prints out the results.

In [ ]:
# Example 5: Ad hoc prediction

# Load X-values
Flower_0 = petalX[[0]]

# Model predicts Y-value
petalPred = petalModel.predict(Flower_0)

# Print out the results
print(f"Flower_0 has X-values: {Flower_0}")
print(f"Model predicts its petal length is: {petalPred}")
print(f"Its actual petal length is: {petalDF['petal_length'].values[0]}")

If your code is correct you should see something similar to the following output:
~~~text
1/1 [==============================] - 0s 22ms/step
Flower_0 has X-values: [[5.1 3.5 0.2 1. ]]
Model predicts its petal length is: [[3.6087673]]
Its actual petal length is: 1.4
~~~

In this particular example, the predicted `petal_length`(3.6087673 cm) is clearly different from the actual `petal length` (1.4 cm). However, the magnitude of this error is not unexpected. 

In Example 4, the RMSE for `petalModel` was calculated to be 1.8 cm. If you add this RMSE value (1.8 cm) to the actual petal length (1.4 cm), the result is 3.2 cm, which is pretty close to the predicted petal length of 3.6 cm. 

### **Exercise 5: _Ad Hoc_ Prediction**

In the cell below, use your `acidModel` to predict the acidity of `Apple_0`.   

In [ ]:
# Insert your code for Exercise 5 here




If your code is correct you should see something similar to the following output:
~~~text
1/1 [==============================] - 0s 21ms/step
Apple_0 has X-values: [[-3.9700484 -2.5123365 -1.0120087  1.8449004  5.3463297  0.3298398
   1.       ]]
Model predicts its acidity is: [[-1.3125017]]
Its actual acidity is: -0.491590483
~~~

### Lesson Summary

The primary objective of this lesson was to demonstrate how to implement **_Early Stopping_** in the training of neural networks. 

Clearly, the ability to stop training when the loss function on the validation training set reaches a minimum can save a significant amount of time. However, perhaps more importantly, Early Stopping can prevent a neural network from **_overtraining_**. 

Overtraining occurs when the neural network can predict training examples with very high accuracy but cannot generalize to new data. In other words, the neural network starts learning **_specific details_** about the training data. While this improves the model's loss function in the particular training set, it will actually performs worse when presented with new data that it hasn't seen before. 


## **Lesson Turn-in**

When you have completed all of the code cells, and run them in sequential order (the last code cell should be number 12), use the **File --> Print.. --> Save to PDF** to generate a PDF of your JupyterLab notebook. Save your PDF as `Class_03_5.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.